# Data Loading, Storage

## JSON Data

In [35]:
import numpy as np
import pandas as pd
np.random.seed(12345)
import matplotlib.pyplot as plt
plt.rc('figure', figsize=(10, 6))
np.set_printoptions(precision=4, suppress=True)
import os

In [2]:
obj = """
{"name": "Wes",
 "places_lived": ["United States", "Spain", "Germany"],
 "pet": null,
 "siblings": [{"name": "Scott", "age": 30, "pets": ["Zeus", "Zuko"]},
              {"name": "Katie", "age": 38,
               "pets": ["Sixes", "Stache", "Cisco"]}]
}
"""

In [3]:
import json
# (JSON)문자열을 파이썬 형태로 변환
result = json.loads(obj)
result

{'name': 'Wes',
 'places_lived': ['United States', 'Spain', 'Germany'],
 'pet': None,
 'siblings': [{'name': 'Scott', 'age': 30, 'pets': ['Zeus', 'Zuko']},
  {'name': 'Katie', 'age': 38, 'pets': ['Sixes', 'Stache', 'Cisco']}]}

In [4]:
# 파이썬 객체를 JSON 형태로 변환
asjson = json.dumps(result)
asjson

'{"name": "Wes", "places_lived": ["United States", "Spain", "Germany"], "pet": null, "siblings": [{"name": "Scott", "age": 30, "pets": ["Zeus", "Zuko"]}, {"name": "Katie", "age": 38, "pets": ["Sixes", "Stache", "Cisco"]}]}'

In [5]:
siblings = pd.DataFrame(result['siblings'], columns=['name', 'age'])
siblings

,name,age
0,Scott,30
1,Katie,38


pd.read_json은 JSON 배열에 담긴 각 객체를 테이블의 로우로 간주한다. Pandas 데이터를 JSON으로 저장하는 한 가지 방법은 DF.to_json 이나 Series.to_json 등이 있다

## XML and HTML: Web Scraping

Python은 lxml, Beautiful Soup와 같이 HTML, XML 형식의 데이터를 읽고 쓸 수 있는 라이브러리가 많다. 아래는 FDIC의 데이터를 통한 실습

In [6]:
conda install lxml

Solving environment: ...working... done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


In [7]:
# list 형식으로 들어온다.
tables = pd.read_html('examples/fdic_failed_bank_list.html')
failures = tables[0]
failures.head()

,Bank Name,City,ST,CERT,Acquiring Institution,Closing Date,Updated Date
0,Allied Bank,Mulberry,AR,91,Today's Bank,"September 23, 2016","November 17, 2016"
1,The Woodbury Banking Company,Woodbury,GA,11297,United Bank,"August 19, 2016","November 17, 2016"
2,First CornerStone Bank,King of Prussia,PA,35312,First-Citizens Bank & Trust Company,"May 6, 2016","September 6, 2016"
3,Trust Company Bank,Memphis,TN,9956,The Bank of Fayette County,"April 29, 2016","September 6, 2016"
4,North Milwaukee State Bank,Milwaukee,WI,20364,First-Citizens Bank & Trust Company,"March 11, 2016","June 16, 2016"


In [8]:
close_timestamps = pd.to_datetime(failures['Closing Date'])
close_timestamps.dt.year.value_counts()

2010    157
2009    140
2011     92
2012     51
2008     25
2013     24
2014     18
2002     11
2015      8
2016      5
2004      4
2001      4
2007      3
2003      3
2000      2
Name: Closing Date, dtype: int64

In [9]:
# Parsing XML with lxml.objectify

from lxml import objectify

path = 'datasets/mta_perf/Performance_MNR.xml'
parsed = objectify.parse(open(path))

# XML 파일에 대한 루트노드에 대한 참조를 얻음
root = parsed.getroot()

In [10]:
data = []

skip_fields = ['PARENT_SEQ', 'INDICATOR_SEQ',
               'DESIRED_CHANGE', 'DECIMAL_PLACES']

# root.INDICATOR 통하여 모든 indicator xml 엘리먼트를 끄집어 낼 수 있다.
for elt in root.INDICATOR:
    el_data = {}
    for child in elt.getchildren():
        if child.tag in skip_fields:
            continue
        el_data[child.tag] = child.pyval
    data.append(el_data)

In [11]:
perf = pd.DataFrame(data)
perf.head()

,AGENCY_NAME,INDICATOR_NAME,DESCRIPTION,PERIOD_YEAR,PERIOD_MONTH,CATEGORY,FREQUENCY,INDICATOR_UNIT,YTD_TARGET,YTD_ACTUAL,MONTHLY_TARGET,MONTHLY_ACTUAL
0,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at thei...,2008,1,Service Indicators,M,%,95.0,96.9,95.0,96.9
1,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at thei...,2008,2,Service Indicators,M,%,95.0,96.0,95.0,95.0
2,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at thei...,2008,3,Service Indicators,M,%,95.0,96.3,95.0,96.9
3,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at thei...,2008,4,Service Indicators,M,%,95.0,96.8,95.0,98.3
4,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at thei...,2008,5,Service Indicators,M,%,95.0,96.6,95.0,95.8


In [12]:
from io import StringIO
tag = '<a href="http://www.google.com">Google</a>'
root = objectify.parse(StringIO(tag)).getroot()

In [13]:
root

<Element a at 0x25d8a092940>

In [14]:
root.get('href')

'http://www.google.com'

In [15]:
root.text

'Google'

## Binary Data Formats

In [16]:
frame = pd.read_csv('examples/ex1.csv')
frame

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


pickle 직렬화를 사용해 데이터를 이진 형식으로 저장하는 것, 효율적으로 저장이 가능하다. 하지만, 오랜 시간이 지나도 안정적으로 데이터를 저장할 것이라는 보장이 낮다. 따라서, 오래 보관할 필요가 없는 데이터일 경우에만 pickle이 추천된다. Pandas는 HDF5와 Message-pack 두 가지 바이너리 포맷을 지원한다. 

In [17]:
frame.to_pickle('examples/frame_pickle')

In [34]:
pd.read_pickle('examples/frame_pickle')

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [36]:
os.remove('examples/frame_pickle')

### Using HDF5 Format

Hierarchial Data Format의 약자로 계층적 데이터 형식을 의미한다. HDF5는 여러개의 데이터 셋을 저장하고 부가정보를 기록할 수 있다. HDF5는 다양한 압축기술을 사용해서 on the fly(실시간) 압축을 지원하며 반복되는 패턴을 가지는 데이터를 좀더 효율적으로 저장할 수 있다. 메모리에 모두 적재할 수 없는 데이터를 필요한 부분만 효과적으로 읽고 쓸 수 있다. 

In [31]:
frame = pd.DataFrame({'a': np.random.randn(100)})
# Store object in HDFStore, mydata.h5 파일이 생성되었다.
store = pd.HDFStore('mydata.h5')
store['obj1'] = frame
store['obj1_col'] = frame['a']
store

<class 'pandas.io.pytables.HDFStore'>
File path: mydata.h5

In [32]:
store.put('obj2', frame, format='table')
store.select('obj2', where=['index >= 10 and index <= 15'])
store.close()

In [44]:
# os.remove("mydata.h5")

## Interacting with Web APIs, using requests

In [50]:
import requests
url = 'https://api.github.com/repos/pandas-dev/pandas/issues'
resp = requests.get(url)
resp

<Response [200]>

In [52]:
data = resp.json()
data[0]['title']

'TYP: sort_index'

In [54]:
issues = pd.DataFrame(data, columns=['number', 'title',
                                     'labels', 'state'])
issues.head()

,number,title,labels,state
0,46300,TYP: sort_index,[],open
1,46298,"BUG: ""data type 'Int64' not understood""","[{'id': 76811, 'node_id': 'MDU6TGFiZWw3NjgxMQ=...",open
2,46297,BUG: Error writing DataFrame with categorical ...,"[{'id': 76811, 'node_id': 'MDU6TGFiZWw3NjgxMQ=...",open
3,46296,CI: xfail geopandas downstream test on MacOS d...,"[{'id': 48070600, 'node_id': 'MDU6TGFiZWw0ODA3...",open
4,46295,BUG: PandasArray[uint].factorize,[],open


## Interacting with Databases

In [55]:
import sqlite3
query = """
CREATE TABLE test
(a VARCHAR(20), b VARCHAR(20),
 c REAL,        d INTEGER
);"""
con = sqlite3.connect('mydata.sqlite')
con.execute(query)
con.commit()

In [56]:
data = [('Atlanta', 'Georgia', 1.25, 6),
        ('Tallahassee', 'Florida', 2.6, 3),
        ('Sacramento', 'California', 1.7, 5)]
stmt = "INSERT INTO test VALUES(?, ?, ?, ?)"
con.executemany(stmt, data)
con.commit()

In [57]:
cursor = con.execute('select * from test')
rows = cursor.fetchall()
rows

[('Atlanta', 'Georgia', 1.25, 6),
 ('Tallahassee', 'Florida', 2.6, 3),
 ('Sacramento', 'California', 1.7, 5)]

In [58]:
cursor.description
pd.DataFrame(rows, columns=[x[0] for x in cursor.description])

,a,b,c,d
0,Atlanta,Georgia,1.25,6
1,Tallahassee,Florida,2.60,3
2,Sacramento,California,1.70,5


SALAlchemy 프로젝트는 SQL 데이터베이스 간의 일반적인 차이점을 추상화하여 제공한다. 

In [59]:
import sqlalchemy as sqla
db = sqla.create_engine('sqlite:///mydata.sqlite')
pd.read_sql('select * from test', db)

,a,b,c,d
0,Atlanta,Georgia,1.25,6
1,Tallahassee,Florida,2.60,3
2,Sacramento,California,1.70,5


In [62]:
os.remove("mydata.sqlite")

PermissionError: [WinError 32] 다른 프로세스가 파일을 사용 중이기 때문에 프로세스가 액세스 할 수 없습니다: 'mydata.sqlite'